In [812]:
# Step 1 Import
import ccxt as ccxt
import pandas as pd
import pandas_ta as ta
import numpy as np
import math
from numba import njit

In [813]:
# Step 2 Fetch Data
def fetch_ohlcv_info(exchange: ccxt.binance, symbol, timeframe, limit) -> pd.DataFrame:
    # Fetch 10,000 candles in chunks of 1500 candles
    num_candles_to_fetch = limit
    candles_per_request = 1000
    start_index = 0
    dataframe = pd.DataFrame()

    while start_index < num_candles_to_fetch:
        # Determine the end index for this request
        end_index = min(start_index + candles_per_request,
                        num_candles_to_fetch)

        # Fetch candles for this chunk
        candles = exchange.fetch_ohlcv(
            symbol, timeframe, limit=candles_per_request, since=None)

        # Convert candles to DataFrame
        df = pd.DataFrame(candles, columns=[
                          'date', 'open', 'high', 'low', 'close', 'volume'])
        df['date'] = pd.to_datetime(df['date'], unit='ms')
        # Append the candles to the DataFrame
        dataframe = pd.concat([dataframe, df], ignore_index=True)

        # Update start index for the next request
        start_index += candles_per_request
    return dataframe

exchange = ccxt.binance()
symbol = "BTC/USDT:USDT"
df = fetch_ohlcv_info(exchange,symbol,"1m",2000)

In [814]:
from numba import njit
import numpy as np

@njit
def calculate_trend_direction_jit(index, close, periods) -> tuple:
    if index >= periods[-1]:
        devMultiplier = 2.0
        highestPearsonR = -np.inf
        detectedPeriod = 0
        detectedSlope = 0
        detectedIntrcpt = 0
        detectedStdDev = 0
        for period in periods:
            stdDev, pearsonR, slope, intercept = calc_dev_jit(period, close, index)
            if pearsonR > highestPearsonR:
                highestPearsonR = pearsonR
                detectedPeriod = period
                detectedSlope = slope
                detectedIntrcpt = intercept
                detectedStdDev = stdDev
        if highestPearsonR == -np.inf:
            raise Exception("Cannot find highest PearsonR")
        startPrice = np.exp(detectedIntrcpt + detectedSlope * (detectedPeriod - 1))
        endPrice = np.exp(detectedIntrcpt)
        trend_direction = endPrice - startPrice
        return trend_direction, detectedPeriod, highestPearsonR

    return 0, 0, 0


@njit
def calc_dev_jit(length, close, index) -> tuple:
    log_source = close[index + 1 - length: index + 1]
    indices = np.arange(1, length + 1)
    sum_x = np.sum(indices)
    sum_xx = np.sum(indices ** 2)
    sum_y = np.sum(log_source)
    sum_yx = np.sum(indices * log_source)

    denominator = length * sum_xx - sum_x ** 2
    if denominator == 0:
        return 0, 0, 0, 0

    slope = (length * sum_yx - sum_x * sum_y) / denominator
    average = sum_y / length
    intercept = average - (slope * sum_x / length) + slope

    deviations = log_source - intercept - slope * (indices - 1)
    sum_dev = np.sum(deviations ** 2)
    if length <= 1 or sum_dev == 0:
        std_dev = 0
    else:
        std_dev = np.sqrt(sum_dev / (length - 1))

    dxt = log_source - average
    dyt = slope * (indices - 1) - (intercept + slope * (length - 1) * 0.5)
    sum_dxx = np.sum(dxt ** 2)
    sum_dyy = np.sum(dyt ** 2)
    sum_dyx = np.sum(dxt * dyt)

    divisor = sum_dxx * sum_dyy
    if divisor <= 0:
        pearson_r = 0
    else:
        pearson_r = sum_dyx / np.sqrt(divisor)

    return std_dev, pearson_r, slope, intercept

def adaptiveTrendFinder_jit(df: pd.DataFrame, periods):
    close = np.log(df['close'].to_numpy())
    trend_directions = np.zeros(len(close))
    detected_periods = np.zeros(len(close))
    highest_pearson_rs = np.zeros(len(close))
    for i in range(len(close)):
        trend_direction, detected_period, highest_pearson_r = calculate_trend_direction_jit(i, close, periods)
        trend_directions[i] = trend_direction
        detected_periods[i] = detected_period
        highest_pearson_rs[i] = highest_pearson_r
    df['trend_direction'] = -trend_directions
    df['detected_period'] = detected_periods
    df['highest_pearson_r'] = highest_pearson_rs
    return df


In [815]:
print(adaptiveTrendFinder_jit(df,[20,30,40,50]))

                    date      open      high       low     close    volume  \
0    2025-01-17 14:52:00  102684.1  102978.4  102654.9  102913.8   709.956   
1    2025-01-17 14:53:00  102913.9  103020.1  102871.2  102917.3   423.617   
2    2025-01-17 14:54:00  102917.3  102980.0  102854.9  102972.9   237.167   
3    2025-01-17 14:55:00  102972.9  103059.1  102966.9  103014.5   371.084   
4    2025-01-17 14:56:00  103014.5  103301.0  103014.5  103249.3  1153.999   
...                  ...       ...       ...       ...       ...       ...   
1995 2025-01-18 07:27:00  103228.0  103228.0  103160.0  103169.6    78.790   
1996 2025-01-18 07:28:00  103169.6  103179.7  103160.1  103179.0    51.575   
1997 2025-01-18 07:29:00  103179.1  103179.1  103150.0  103150.1    58.174   
1998 2025-01-18 07:30:00  103150.1  103150.1  103090.0  103102.9   105.532   
1999 2025-01-18 07:31:00  103102.8  103117.1  103052.1  103052.1    61.418   

      trend_direction  detected_period  highest_pearson_r  
0  